In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [7]:
week7_link = 'http://www.footballlocks.com/nfl_odds_week_7.shtml#Closing%20NFL%20Odds%20Week%201,%202007'

#get html text
week7_html = requests.get(week7_link).text

soup = BeautifulSoup(week7_html, 'html.parser')

#want just table row data
tr = soup.find_all("tr")

len(tr)

#start of table
print(tr[51])

#last entry in table
print(tr[252])

<tr>
<td>10/19 8:25 ET</td>
<td>Kansas City</td>
<td>-3</td>
<td>At Oakland</td>
<td>46.5</td>
<td>-$155 +$135</td>
</tr>
<tr>
<td>10/23 8:30 ET</td>
<td>At Dallas</td>
<td>-3</td>
<td>NY Giants</td>
<td>43.5</td>
<td>-$170 +$150</td>
</tr>


In [8]:
#want to 253 because it does not include last one
table = tr[51:253]

type(table[-1])

type(table)

string_table = [str(i) for i in table]

type(string_table[0])

week7_df = pd.DataFrame(string_table)

week7_df.columns = ['all_data']

week7_df.head()

,all_data
0,<tr>\n<td>10/19 8:25 ET</td>\n<td>Kansas City<...
1,<tr>\n<td>10/22 1:00 ET</td>\n<td>Tennessee</t...
2,<tr>\n<td>10/22 1:00 ET</td>\n<td>Jacksonville...
3,<tr>\n<td>10/22 4:25 ET</td>\n<td>At Pittsburg...
4,<tr>\n<td>10/22 1:00 ET</td>\n<td>At Minnesota...


In [9]:
#want to clean the date data

dates = [x.split('td')[1] for x in week7_df['all_data']]

cleaned_dates = []
for x in dates:
    str1 = x.replace(">", "").replace("<", "")
    final_str = str1[:-1]
    cleaned_dates.append(final_str)

week7_df['date'] = cleaned_dates

In [10]:
#pull out favorites

week7_df['cleaned_favorites'] = week7_df['all_data'].map(lambda x: x.split('td')[3].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create underdog column

week7_df['cleaned_underdogs'] = week7_df['all_data'].map(lambda x: x.split('td')[7].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create spread column
week7_df['cleaned_spread'] = week7_df['all_data'].map(lambda x: x.split('td')[5].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create over_under column
week7_df['cleaned_over_under'] = week7_df['all_data'].map(lambda x: x.split('td')[9].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create money_odds column and clean data
money_odds = []
for x in week7_df['all_data']:
    try:
        money_odds.append(x.split('td')[11].replace('<', '').replace('>', '').replace('/', ''))
    except:
        money_odds.append(np.nan)
        
week7_df['money_odds'] = money_odds

#input week number for each game
week7_df['week_number'] = '7'

In [11]:
#want to input year for each game. on lines we want to decrease the count by one, len(week1_df.all_data[x].split('td')) = 3
year = 2018
year_list = []
for x in week7_df['all_data']:
    if len(x.split('td')) > 3:
        year_list.append(year)
    else:
        year_list.append(year)
        year -= 1

week7_df['year'] = year_list

In [12]:
#create a final dataframe for week 7
week7_final = week7_df[['date', 'year', 'week_number', 'cleaned_favorites', 'cleaned_underdogs', 'cleaned_spread', 
                        'cleaned_over_under', 'money_odds']]
week7_final.head()

,date,year,week_number,cleaned_favorites,cleaned_underdogs,cleaned_spread,cleaned_over_under,money_odds
0,10/19 8:25 ET,2018,7,Kansas City,At Oakland,-3,46.5,-$155 +$135
1,10/22 1:00 ET,2018,7,Tennessee,At Cleveland,-6,42.5,-$250 +$210
2,10/22 1:00 ET,2018,7,Jacksonville,At Indianapolis,-3,41.5,-$165 +$145
3,10/22 4:25 ET,2018,7,At Pittsburgh,Cincinnati,-4,40,-$210 +$180
4,10/22 1:00 ET,2018,7,At Minnesota,Baltimore,-5,38,-$240 +$200


In [13]:
#week7_final to pickle
week7_final.to_pickle('week7_final.pkl')